# Test simulations on a TKD mesh

The following notebook outlines a sample simulation of the perfusion and gas exchange model on a periodic mesh mimicking an acinus as a collection of tetrakaidecahedral alveoli. We consider an infinite packing of tetrakaidecahedra, each one of which represents a single, $200 \, \mu \text{m}$ alveolus.

Import necessary packages and add the working directory to the system path.

In [1]:
import sys
import os
import dolfin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
sys.path.append(os.getcwd()[:-6])

Import the model modules.

In [2]:
from src.model import PerfusionGasExchangeModel
from src.params import params

Set a target folder for `vtk` files.

In [3]:
folder = "tkd"
path = os.path.join("../raw-data", folder)

Instance the model and run simulation.

In [4]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.import_mesh(
    os.path.join("../raw-data", "TKD.xml"), type="xml"
)
model.sim_p(save=True, meshtype="tkd")
x = model.sim_sbst(hb=False, save=True)
#solution = model.sim_sbst(hb=True, save=True, guess=x)
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries

Solving linear variational problem.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 2.040e+09 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.130e-06 (tol = 1.000e-08) r (rel) = 5.537e-16 (tol = 1.000e-09)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
